In [161]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report
from torch import optim, nn
import wandb
from tqdm import tqdm

wandb.login()

True

In [162]:
config = {
    "learning_rate": 0.005,
    "architecture": "NN",
    "dataset": "static_1.1",
    "epochs": 50,
    "classes": 2,
    "batch_size": 64,
    "num_layers": 4,
    "hidden_size": 128,
    "dropout_prob": 0.1,
    "input_size": 89,
    "output_size": 1,
    "optimizer": "Adam",
    "loss": "CrossEntropyLoss",
    "activation": "ReLU",
    "decrease_size": True,
}

In [163]:
def model_pipeline(hyperparameters):
    with wandb.init(project="leaguify", config=hyperparameters):
        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config
        # make the model, data, and optimization problem
        model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        # and use them to train the model
        train(model, train_loader, val_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, val_loader)
        test(model, test_loader)

    return model

In [164]:
def get_activation(name):
    if name == 'ReLU':
        return nn.ReLU()
    elif name == 'LeakyReLU':
        return nn.LeakyReLU()
    elif name == 'ELU':
        return nn.ELU()
    elif name == 'SELU':
        return nn.SELU()
    elif name == 'Tanh':
        return nn.Tanh()
    elif name == 'Sigmoid':
        return nn.Sigmoid()
    else:
        raise ValueError(f'Activation {name} not supported')

In [165]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.transform = transform
        self.target_transform = target_transform
        self.print_statistics()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

    def print_statistics(self):
        print(f'Number of samples: {len(self.data)}')
        print(f'Number of features: {len(self.data[0])}')
        print(f'Number of classes: {len(np.unique(self.labels.cpu().numpy()))}')
        print(f'Number of samples per class: {np.bincount(self.labels.cpu().numpy())}')

In [166]:
def make(config):
    train_data, val_data = get_train_data()
    train_loader = make_loader(train_data, batch_size=config.batch_size)
    val_loader = make_loader(val_data, batch_size=config.batch_size)
    test_loader = make_loader(get_test_data(), batch_size=config.batch_size)
    activation = get_activation(config.activation)

    model = NeuralNetwork(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob,
                          config.classes, activation, config.decrease_size).to(device)
    criterion = nn.CrossEntropyLoss()
    if config.optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=config.learning_rate)
    else:
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    return model, train_loader, val_loader, test_loader, criterion, optimizer

In [167]:
def get_train_data(val_split=0.8):
    dataset = StaticDataset('../data/processed/train_static.npy')
    train_len = int(len(dataset) * val_split)
    val_len = len(dataset) - train_len
    print(f'train_len: {train_len}, val_len: {val_len}')
    return torch.utils.data.random_split(dataset, [train_len, val_len])

In [168]:
def get_test_data():
    return StaticDataset('../data/processed/test_static.npy')

In [169]:
def make_loader(dataset, batch_size=64):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [170]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
print(f"Using {device} device")

PyTorch version: 2.1.0+cu121
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:09:35_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
**********
CUDNN version: 8801
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 2080
Using cuda device


In [171]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, output_size=1, activation=nn.ReLU(),
                 decrease_size=False):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential()
        self.linear_relu_stack.append(nn.Linear(input_size, hidden_size))
        for i in range(num_layers - 1):
            if decrease_size:
                next_hidden_size = int(self.hidden_size // 2)
            else:
                next_hidden_size = self.hidden_size
            self.linear_relu_stack.append(self.dropout)
            self.linear_relu_stack.append(nn.BatchNorm1d(self.hidden_size))
            self.linear_relu_stack.append(nn.Linear(self.hidden_size, next_hidden_size))
            self.linear_relu_stack.append(activation)
            self.hidden_size = next_hidden_size
        self.linear_relu_stack.append(nn.Linear(self.hidden_size, self.output_size))
        self.linear_relu_stack.append(nn.Sigmoid())

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [172]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)
    example_count = 0
    batch_count = 0
    for epoch in tqdm(range(config.epochs)):
        model.train()
        for _, (matches, labels) in enumerate(train_loader):
            matches, labels = matches.to(device), labels.to(device)
            loss = train_batch(matches, labels, model, optimizer, criterion)
            example_count += len(matches)
            batch_count += 1
            if ((batch_count + 1) % 25) == 0:
                train_log(loss, example_count, epoch)
                test(model, val_loader)

In [173]:
def train_batch(matches, labels, model, optimizer, criterion):
    matches, labels = matches.to(device), labels.to(device)

    # Forward pass ➡
    outputs = model(matches)
    loss = criterion(outputs, labels)

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [174]:
def train_log(loss, example_count, epoch):
    wandb.log({"epoch": epoch, "loss": loss}, step=example_count)
    print(f"Loss after {str(example_count).zfill(5)} examples: {loss:.3f}")

In [175]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        y_pred = []
        y_true = []
        for matches, labels in test_loader:
            matches, labels = matches.to(device), labels.to(device)
            outputs = model(matches)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(classification_report(y_pred, y_true))

        print(f"Accuracy of the model on the {total} " +
              f"test matches: {correct / total:%}")

        wandb.log({"test_accuracy": correct / total})

In [176]:
model = model_pipeline(config)

Number of samples: 15960
Number of features: 90
Number of classes: 2
Number of samples per class: [7591 8369]
train_len: 12768, val_len: 3192
Number of samples: 3990
Number of features: 90
Number of classes: 2
Number of samples per class: [1902 2088]
NeuralNetwork(
  (dropout): Dropout(p=0.1, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=89, out_features=128, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=64, out_features=32, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.1, inplace=False)
    (10): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): 

  0%|          | 0/50 [00:00<?, ?it/s]

Loss after 01536 examples: 0.643
              precision    recall  f1-score   support

           0       0.60      0.64      0.62      1416
           1       0.70      0.66      0.68      1776

    accuracy                           0.65      3192
   macro avg       0.65      0.65      0.65      3192
weighted avg       0.66      0.65      0.66      3192

Accuracy of the model on the 3192 test matches: 65.476190%
Loss after 03136 examples: 0.631
              precision    recall  f1-score   support

           0       0.65      0.62      0.64      1570
           1       0.65      0.67      0.66      1622

    accuracy                           0.65      3192
   macro avg       0.65      0.65      0.65      3192
weighted avg       0.65      0.65      0.65      3192

Accuracy of the model on the 3192 test matches: 64.974937%
Loss after 04736 examples: 0.703
              precision    recall  f1-score   support

           0       0.63      0.65      0.64      1456
           1       0

  2%|▏         | 1/50 [00:02<01:54,  2.33s/it]

              precision    recall  f1-score   support

           0       0.66      0.64      0.65      1537
           1       0.68      0.69      0.68      1655

    accuracy                           0.67      3192
   macro avg       0.67      0.67      0.67      3192
weighted avg       0.67      0.67      0.67      3192

Accuracy of the model on the 3192 test matches: 66.635338%
Loss after 14304 examples: 0.630
              precision    recall  f1-score   support

           0       0.67      0.65      0.66      1570
           1       0.67      0.70      0.68      1622

    accuracy                           0.67      3192
   macro avg       0.67      0.67      0.67      3192
weighted avg       0.67      0.67      0.67      3192

Accuracy of the model on the 3192 test matches: 67.167920%
Loss after 15904 examples: 0.619
              precision    recall  f1-score   support

           0       0.57      0.69      0.63      1247
           1       0.77      0.67      0.72      1945

...

Loss after 627168 examples: 0.626
              precision    recall  f1-score   support

           0       0.61      0.63      0.62      1462
           1       0.68      0.66      0.67      1730

    accuracy                           0.65      3192
   macro avg       0.65      0.65      0.65      3192
weighted avg       0.65      0.65      0.65      3192

Accuracy of the model on the 3192 test matches: 64.912281%
Loss after 628768 examples: 0.474
              precision    recall  f1-score   support

           0       0.68      0.62      0.65      1671
           1       0.62      0.69      0.65      1521

    accuracy                           0.65      3192
   macro avg       0.65      0.65      0.65      3192
weighted avg       0.65      0.65      0.65      3192

Accuracy of the model on the 3192 test matches: 65.006266%
Loss after 630368 examples: 0.523
              precision    recall  f1-score   support

           0       0.61      0.65      0.63      1427
           1     

100%|██████████| 50/50 [01:55<00:00,  2.30s/it]

Loss after 638368 examples: 0.597
              precision    recall  f1-score   support

           0       0.65      0.63      0.64      1554
           1       0.66      0.68      0.67      1638

    accuracy                           0.66      3192
   macro avg       0.66      0.66      0.66      3192
weighted avg       0.66      0.66      0.66      3192

Accuracy of the model on the 3192 test matches: 65.789474%


              precision    recall  f1-score   support

           0       0.65      0.64      0.64      1530
           1       0.67      0.68      0.67      1662

    accuracy                           0.66      3192
   macro avg       0.66      0.66      0.66      3192
weighted avg       0.66      0.66      0.66      3192

Accuracy of the model on the 3192 test matches: 65.852130%
              precision    recall  f1-score   support

           0       0.67      0.65      0.66      1969
           1       0.67      0.69      0.68      2021

    accuracy                           0.67      3990
   macro avg       0.67      0.67      0.67      3990
weighted avg       0.67      0.67      0.67      3990

Accuracy of the model on the 3990 test matches: 66.791980%


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▇▅▃▇▆▅█▅▃▆▅▄▄▅▅█▆▇▇▇▂▅▂▄▃▅▂▅█▄▆▄▂▃▃▄▁▅▃▇
test_accuracy,▄▇██▇▇▇▇█▆▅▅▅▅▆▄▃▄▅▃▃▄▂▄▄▂▃▃▂▂▁▂▁▄▄▂▁▃▃▃
epoch,49
loss,0.5968
test_accuracy,0.66792
